# Implementing Principal Component Analysis (PCA)
We can implement PCA using pure numpy. Before we get started, let's import some necessary packages.  We will need numpy and numpy.linalg

In [1]:
import numpy as np
import numpy.linalg as la

Then we need some data.  To begin with, we'll just use some arbitrary, made-up data.  Note, though, that the columns have to be linearly independent or we could end up with a singular matrix.  Think of it like this, if we do have data with linearly dependent columns, then that means that the column data only differs by a scalar factor.  Because of this, that column contributes no information above what is contributed by the other column and, so, we can discard it.  When creating data, it's easy to create linearly dependent columns.  In the real world, though, it's rare to have to worry about this.  So, let's start with this data:

In [2]:
X = np.array([
    [24, 36, 15],
    [44, 25, 63],
    [37, 84, 29],
    [11, 31, 26]
    ])


Our data is a 4 x 3 matrix, so we have N (number of samples) is 4 and M (dimensions or number of features) is 3.  The columns are linearly independent, so we are all set!

Since we have 3 dimensions, it's not out of the question that we could plot this data.  But, for our purposes, let's suppose that we want to visualize this data in two dimensions instead.  So we are going to set our dimensions in a variable:

In [3]:
d = 2

So our plan is to project our three dimensional data into a two dimensional subspace such that the variance is maximized along the two principal components which span the subspace.  The first thing we need is a covariance matrix that represents our data.

In [4]:
C = np.cov(X, rowvar=False)

Note that we use the `rowvar=False` flag.  By default, `numpy.cov` expects your features to be in rows and data points in colums.  This seems odd, so it's important to remember this about `numpy.cov`.

Given our covariance matrix $\mathbf{C}$, we now compute the eigendecomposition of $\mathbf{C}$.

In [20]:
w,v = la.eig(C)

`numpy.linalg.eig` computes the eigendecomposition of a matrix and returns the eigenvalues and eigenvectors respectively.  For PCA, we want to reorder the eigenvalues from highest to lowest and then sort the eigenvectors to match the sorted eigenvalues.

In [6]:
i = w.argsort()[::-1]
W = v[:, i]

Now it's time to project to our lower-dimensional sub-space.  In this case, we are projecting from three dimensions to `d = 2` dimensions.  Remember, before projecting, we have to mean center our data by subtracting the mean of each feature column from each datapoint.  The equation for projecting is:

$$\mathbf{z} = \mathbf{W}^T(\mathbf{x} - \mathbf{m})$$

In [22]:
Z = (X - np.mean(X, axis=0)).dot(W[:, 0:d])
print(Z)

[[  0.26039012 -18.99573917]
 [ 28.88809528  25.2195718 ]
 [-38.58708635  13.84373353]
 [  9.43860095 -20.06756616]]


Let's compare this to scikit learn's PCA implementation!

In [9]:
from sklearn.decomposition import PCA

pca = PCA(n_components=d)
Z_pca = pca.fit_transform(X)
print(Z_pca)

[[ -0.26039012 -18.99573917]
 [-28.88809528  25.2195718 ]
 [ 38.58708635  13.84373353]
 [ -9.43860095 -20.06756616]]


The absolute values match up, but the data seems to be a reflection of itself.  I'm not sure why this is, but PCA is, indeed, working in both cases and they're equivalent.
For completeness, the code above is codified in a python class below and shown to give the same results.

In [15]:
class MyPCA:

    def __init__(self, n_components):
        self.n_components = n_components

    def fit_transform(self, X):
        # C is the covariance matrix of X
        C = np.cov(X, rowvar=False)

        # Perform an eigendecomposition of C to obtain the eigenvalues
        # and eigenvectors. w contains the eigenvalues and v contains
        # the eigenvectors
        w, v = la.eig(C)

        # Sort the eigenvalues highest to lowest and store the ordered
        # eigenvectors in W
        i = w.argsort()[::-1]
        W = v[:, i]

        # Store the proportion of variance explained stats
        ev = np.sort(w)[::-1][0:self.n_components]
        evr = ev / np.sum(w)
        self.explained_variance_ = ev
        self.explained_variance_ratio_ = evr
        self.components_ = W[:, 0:self.n_components].T

        # Before projecting, mean center our data by subtracting the
        # mean by columns (axis=0).  Then project onto the subspace
        # represented by the n_components eigenvectors
        return (X - np.mean(X, axis=0)).dot(W[:, 0:self.n_components])

mypca = MyPCA(n_components=d)
X_mypca = mypca.fit_transform(X)

print('X_mypca:\n{}'.format(X_mypca))

print('pca explained_variance: {}'.format(pca.explained_variance_))
print('pca explained variance ratio: {}'.format(pca.explained_variance_ratio_))

print('mypca explained_variance: {}'.format(mypca.explained_variance_))
print('mypca explained variance ratio: {}'.format(mypca.explained_variance_ratio_))


X_mypca:
[[  0.26039012 -18.99573917]
 [ 28.88809528  25.2195718 ]
 [-38.58708635  13.84373353]
 [  9.43860095 -20.06756616]]
pca explained_variance: [ 603.16006817  397.80526964]
pca explained variance ratio: [ 0.58548572  0.38614841]
mypca explained_variance: [ 804.21342423  530.40702619]
mypca explained variance ratio: [ 0.58548572  0.38614841]
